# Traffic Sign Recognition

## Import Libraries

In [ ]:
! pip install neptune-client --quiet
! pip install livelossplot --quiet

In [ ]:
CUDA_LAUNCH_BLOCKING = "1"
import numpy as np
import pandas as pd
import os
import collections

from PIL import Image
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from matplotlib import style
style.use('fivethirtyeight')

import random
seed = 42
np.random.seed(seed)

In [ ]:
from pytorch_lightning.loggers.neptune import NeptuneLogger

neptune_logger = NeptuneLogger(
    api_key="...",
    project_name="...")

In [ ]:
root_dir = '../input/gtsrb-german-traffic-sign'
train_dir = '../input/gtsrb-german-traffic-sign/Train'
test_dir = '../input/gtsrb-german-traffic-sign/Test'

# To resize the images to 32x32x3
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
N_CHANNELS = 3 # R-G-B

In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

## Visualize Class Distribution (Training Dataset)

In [ ]:
folders = os.listdir(train_dir)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_dir + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])
    
# Sorting the dataset on the basis of number of images in each class
zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [list(tuple) for tuple in tuples]

# Plotting the number of images in each class
plt.figure(figsize=(21,10))  
plt.bar(class_num, train_number)
plt.xticks(class_num, rotation='vertical')
plt.show()

## Visualize Random Samples (Testing Dataset)

In [ ]:
# Visualizing 25 random images from test data
import random
from matplotlib.image import imread

test = pd.read_csv(root_dir + '/Test.csv')
imgs = test["Path"].values

plt.figure(figsize=(25,25))

for i in range(1,26):
    plt.subplot(5,5,i)
    random_img_path = root_dir + '/' + random.choice(imgs)
    rand_img = imread(random_img_path)
    plt.imshow(rand_img)
    plt.grid(b=None)
    plt.xlabel(rand_img.shape[1], fontsize = 20) # width of image
    plt.ylabel(rand_img.shape[0], fontsize = 20) # height of image

## Dataset & Data Augmentation

In [ ]:
class GTSRBDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        """Initializes a dataset containing images and labels."""
        super().__init__()
        
        self.transform = transform
        self.images = images 
        self.labels = labels
                
    def __len__(self):
        """Returns the size of the dataset."""
        return len(self.labels)

    def __getitem__(self, index):
        """Returns the index-th data item of the dataset."""
        image = self.images[index]
        if self.transform:
            image = self.transform(image)
        return image, self.labels[index]

In [ ]:
def load_dataset(root_dir, train=True):
    csv_file = ["Test.csv", "Train.csv"][train]
    csv = pd.read_csv(root_dir + '/' + csv_file)
    image_paths = csv["Path"].values
    image_labels = csv["ClassId"].values
    images = []
    labels = []
    
    for image_path, image_label in zip(image_paths, image_labels):
        image = Image.open(root_dir + '/' + image_path).convert('RGB')
        images.append(image)
        labels.append(image_label)
    
    return images, image_labels


full_train_set = load_dataset(root_dir, train=True)

In [ ]:
validation_size = 0.4
train_images, validation_images, train_labels, validation_labels = train_test_split(full_train_set[0], 
                                                                                    full_train_set[1], 
                                                                                    test_size=validation_size, 
                                                                                    random_state=seed,
                                                                                    stratify=full_train_set[1])
print(f"Trainset length: {len(train_images)}")
print(f"Validation set length: {len(validation_images)}")

In [ ]:
# Check that the train set and the validation set have the same distribution of classes
import collections
train_set_frequency = collections.Counter(sorted(train_labels))
print({k: v/len(train_labels) for k, v in dict(train_set_frequency).items()})
frequency = collections.Counter(sorted(validation_labels))
print({k: v/len(validation_labels) for k, v in dict(frequency).items()})

In [ ]:
def show_images(img):
    plt.figure(figsize=(14, 6))
    npimg = img.numpy() * 0.5 + 0.5
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.grid(b=None)
    plt.show()

def show_sample_data_loader(loader):
    dataiter = iter(loader)
    images, labels = dataiter.next()
    show_images(torchvision.utils.make_grid(images, nrow=5))
    print('\n'.join('%s' % classes[label] for label in labels.tolist()))

### Data Augmentation (Validation/Test Set)

In [ ]:
# source: https://androidkt.com/pytorch-image-augmentation-using-transforms/
class AddGaussianNoise(object):
    '''
    Add gaussian noise
    
    Params:
        mean: mean of the gaussian distribution
        std:  standard deviation of the gaussian distribution
    '''
    def __init__(self, mean=0.0, std=1.0):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

In [ ]:
test_transform = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
                                    transforms.ToTensor(), 
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

validation_set = GTSRBDataset(validation_images, validation_labels, transform=test_transform)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=10, shuffle=True, num_workers=0)

show_sample_data_loader(validation_loader)

### Data Augmentation (Train Set)

We apply different random transformations to the train set images that should create additional coherent and plausible road sign images to train from. They may very well be found as such in the test set (eg: we should not apply a random horizontal flip because an important part of road signs won't have any meaning such as a reverse speed limit (50km/h) that won't be readable). We thus opted for a random combination of:
- a random rotation and shear to cope for the different angles of view the image could have been taken from.
- a random crop of 30% of the image size to cope for the fact that road signs may not always be completely present on the photographs.
- a random color jitter to cope for reflection, ambient light, weather change, and potential lack of image quality coming from the camera.
- 

In [ ]:
class RandomComposeTransform(object):
    '''
    Randomly choose among a subset of transforms to apply
    
    Params:
        transform_list: a list of transforms to apply in the given order
    '''
    def __init__(self, transform_list):
        self.transform_list = transform_list
        
    def __call__(self, tensor):
        transform_list_len = len(self.transform_list) 
        random_sorted_indexes = sorted(random.sample(range(transform_list_len), k=random.randint(0, transform_list_len)))
        if not random_sorted_indexes:
            return transforms.RandomAffine(degrees=0)(tensor) # identity transform <-> applies no transformation
        return transforms.Compose([self.transform_list[i] for i in random_sorted_indexes])(tensor)

In [ ]:
crop_fraction = 0.3
random_crop_transform = transforms.Compose([transforms.Resize((round(IMAGE_HEIGHT * (1 + crop_fraction)), round(IMAGE_WIDTH * (1 + crop_fraction)))),
                                            transforms.RandomCrop((IMAGE_HEIGHT, IMAGE_WIDTH))])
random_compose_transforms = RandomComposeTransform([transforms.RandomAffine(degrees=10, shear=40),
                                                    random_crop_transform,
                                                    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05)
                                                    #AddGaussianNoise(0.1, 0.05),
                                                   ])
train_transform = transforms.Compose([#transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
                                      random_compose_transforms,
                                      #AddGaussianNoise(0.1, 0.05),
                                      transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = GTSRBDataset(train_images, train_labels, transform=train_transform) # training dataloader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True, num_workers=0)

show_sample_data_loader(train_loader)

## Train - Validation Data Loader

In [ ]:
#train_labels_frequency = torch.tensor(list(collections.Counter(train_labels).values()))
#weighted_sampler = WeightedRandomSampler(train_labels_frequency[torch.tensor(train_labels)], len(train_labels), replacement=True)

# Training dataloader
train_set = GTSRBDataset(train_images, train_labels, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, num_workers=50)
#train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=False, num_workers=50, sampler=weighted_sampler)

# Validation dataloader
validation_set = GTSRBDataset(validation_images, validation_labels, transform=test_transform)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=100, shuffle=False, num_workers=50)

In [ ]:
#print(next(iter(train_loader)).shape)
batch_iter = iter(train_loader)
image, labels = batch_iter.next()

print(labels.tolist())
train_batch_frequency = collections.Counter(sorted(labels.tolist()))
print({k: v/100 for k, v in dict(train_batch_frequency).items()})

## Model

In [ ]:
# Compute the output size of a conv layer
def outputSize(in_size, kernel_size, stride=1, padding=1):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)  

In [ ]:
outputSize(13, 2, 2, 1)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
input_dim = input_dim
BN1 = nn.BatchNorm2d(input_dim)
conv1 = nn.Conv2d(input_dim, conv1_output_channels, kernel_size=3, stride=1, padding=0)
pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
BN2 = nn.BatchNorm2d(conv1_output_channels)
conv2 = nn.Conv2d(conv1_output_channels, conv2_output_channels, kernel_size=3, stride=1, padding=0)
BN3 = nn.BatchNorm2d(conv2_output_channels)
conv3 = nn.Conv2d(conv2_output_channels, last_conv_output_channels, kernel_size=3, stride=1, padding=0)

BN4 = nn.BatchNorm2d(last_conv_output_channels)
fc1_input_channels = last_conv_output_channels * last_conv_output_dim * last_conv_output_dim
fc1 = nn.Linear(fc1_input_channels, fc1_input_channels)
fc2 = nn.Linear(fc1_input_channels, output_dim)

dataiter = iter(train_loader)
images, labels = dataiter.next()
x = images
print(x.shape)
# Block 1
x = BN1(x)
x = F.relu(conv1(x))
x = pool(x)
print(x.shape)
# Block 2
x = BN2(x)
x = F.relu(conv2(x))
x = pool(x)
print(x.shape)
# Block 3
x = BN3(x)
x = F.relu(conv3(x))
x = pool(x)
print(x.shape)
# Block 4
x = BN4(x)
x = x.view(-1, last_conv_output_channels * last_conv_output_dim * last_conv_output_dim)
x = F.relu(fc1(x))
x = fc2(x)
print(x.shape)

In [ ]:
from pytorch_lightning.loggers.neptune import NeptuneLogger
from livelossplot import PlotLosses

neptune_logger = NeptuneLogger(
    api_key="...",
    project_name="...")
#liveplot = PlotLosses()

In [ ]:
pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
testmodel = EfficientNet.from_pretrained('efficientnet-b4')
print(testmodel)

In [ ]:
input_dim = 3 
conv1_output_channels = 32
conv2_output_channels = 64
last_conv_output_channels = 128
last_conv_output_dim = 4
output_dim = len(classes)
learning_rate = 0.001
max_epochs = 100
criterion = nn.CrossEntropyLoss()

class CNN(pl.LightningModule): #nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN, self).__init__()
        
        # self.input_dim = input_dim
        # self.BN1 = nn.BatchNorm2d(input_dim)
        # self.conv1 = nn.Conv2d(input_dim, conv1_output_channels, kernel_size=3, stride=1, padding=0)
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        # self.BN2 = nn.BatchNorm2d(conv1_output_channels)
        # self.conv2 = nn.Conv2d(conv1_output_channels, conv2_output_channels, kernel_size=3, stride=1, padding=0)
        # self.BN3 = nn.BatchNorm2d(conv2_output_channels)
        # self.conv3 = nn.Conv2d(conv2_output_channels, last_conv_output_channels, kernel_size=3, stride=1, padding=0)
        
        # self.BN4 = nn.BatchNorm2d(last_conv_output_channels)
        # fc1_input_channels = last_conv_output_channels * last_conv_output_dim * last_conv_output_dim
        self.base = EfficientNet.from_pretrained('efficientnet-b4')
        for param in self.base.parameters():
            param.requires_grad = False
        
        num_features = self.base._fc.in_features
        self.base._fc = nn.Linear(num_features, output_dim)
        # self.fc1 = nn.Linear(fc1_input_channels, fc1_input_channels)
        # self.fc2 = nn.Linear(fc1_input_channels, output_dim)

    def forward(self, x):
        # # Block 1
        # x = self.BN1(x)
        # x = F.relu(self.conv1(x))
        # x = self.pool(x)
        
        # # Block 2
        # x = self.BN2(x)
        # x = F.relu(self.conv2(x))
        # x = self.pool(x)

        # # Block 3
        # x = self.BN3(x)
        # x = F.relu(self.conv3(x))
        # x = self.pool(x)
        
        # # Block 4
        # x = self.BN4(x)
        # x = x.view(-1, last_conv_output_channels * last_conv_output_dim * last_conv_output_dim)
        # x = F.relu(self.fc1(x))
        # x = self.fc2(x)
        x = self.base(x)
        return x
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.forward(x)
        loss = criterion(y_hat, y)
        #self.log('train_loss', loss)
        y_hat = torch.argmax(y_hat, dim=1)#.cpu()
        #accuracy = accuracy_score(y.cpu(), y_hat)
        #balanced_accuracy = balanced_accuracy_score(y.cpu(), y_hat)
        accuracy = torch.sum(y == y_hat).item() / len(y)
        #self.log('train_accuracy', accuracy)
        logs = {'train_loss': loss, 'train_accuracy': accuracy}#, 'train_balanced_accuracy': balanced_accuracy}
        self.log_dict(logs, on_epoch=True)
        #liveplot.update(logs)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.forward(x)
        loss = criterion(y_hat, y)
        #self.log('val_loss', loss)
        y_hat = torch.argmax(y_hat, dim=1)#.cpu()
        #accuracy = accuracy_score(y.cpu(), y_hat)
        #balanced_accuracy = balanced_accuracy_score(y.cpu(), y_hat)
        accuracy = torch.sum(y == y_hat).item() / len(y)
        #self.log('val_accuracy', accuracy)
        logs = {'val_loss': loss, 'val_accuracy': accuracy}#, 'val_balanced_accuracy': balanced_accuracy}
        self.log_dict(logs)
        #liveplot.update(logs)
        return loss
    
    def test_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.forward(x)
        loss = criterion(y_hat, y)
        #self.log('test_loss', loss)
        y_hat = torch.argmax(y_hat, dim=1)#.cpu()
        #accuracy = accuracy_score(y, y_hat)
        #balanced_accuracy = balanced_accuracy_score(y, y_hat)
        accuracy = torch.sum(y == y_hat).item() / len(y)
        #self.log('test_accuracy', accuracy)
        logs = {'test_loss': loss, 'test_accuracy': accuracy}#, 'test_balanced_accuracy': balanced_accuracy}
        self.log_dict(logs)
        #liveplot.update(logs)
        return loss
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        return optimizer

    
#device = 'cuda:0'
model = CNN(input_dim, output_dim)#.to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
early_stop_callback = EarlyStopping(
   monitor='val_accuracy',#'val_balanced_accuracy',
   min_delta=0.001,
   patience=8,
   verbose=True,
   mode='max'
)

trainer = pl.Trainer(gpus=1, max_epochs=max_epochs, callbacks=[early_stop_callback], logger=neptune_logger)
trainer.fit(model, train_loader, validation_loader)
#liveplot.send()

In [ ]:
test_images, test_labels = load_dataset(root_dir, train=False)
print(f"Trainset shape: {len(full_train_set[0])}")
print(f"Test set shape: {len(test_images)}")

In [ ]:
#full_train_set = GTSRBDataset(full_train_set[0], full_train_set[1], transform=train_transform)
#full_train_loader = torch.utils.data.DataLoader(full_train_set, batch_size=100, shuffle=True, num_workers=50)
test_set = GTSRBDataset(test_images, test_labels, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=50)

In [ ]:
trainer.test(model, test_dataloaders=test_loader)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
model.eval()
pred_labels = []
model.to('cpu')
wrong_dict = {"pred_labels":[], "true_labels":[], "images":[]}
for test_image, test_label in test_set:
    pred_label = torch.argmax(model(test_image.unsqueeze(0))).item()
    pred_labels.append(pred_label)
    
    if pred_label != test_label: # if wrongly classified
        wrong_dict["pred_labels"].append(pred_label)
        wrong_dict["true_labels"].append(test_label)
        wrong_dict["images"].append(np.transpose(test_image, (1, 2, 0)))
    
cf = confusion_matrix(test_labels, pred_labels)

In [ ]:
n_wrong_images = len(wrong_dict)
print(f"Number of wrongly classified test images: {n_wrong_images}")
n_cols = 10
img_size = 2.5
i = 0
plt.figure(figsize=(round(img_size * n_wrong_images / n_cols), img_size * n_cols))
for image in wrong_dict["images"]:
    plt.subplot(n_wrong_images/n_cols, n_cols, i)
    i += 1
    plt.imshow(image) 
    plt.grid(b=None)

In [ ]:
import seaborn as sns
df_cm = pd.DataFrame(cf, index=classes,  columns=classes)
plt.figure(figsize = (20,20))
sns.heatmap(df_cm, annot=True)

## Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, pred_labels))

## Train the Model

In [ ]:
def train(num_epochs, trainloader, testloader):
    epochs_train_loss = []
    epochs_test_loss = []
    epochs_train_accuracy = []
    epochs_test_accuracy = []
    for i in range(num_epochs):
        print(i)
        if i % 1 == 0: # no use...
            with torch.no_grad():
                correct = 0
                total = 0
                tmp_test_loss = []
                for inputs, targets in testloader:
                    outputs = model(inputs.to(device))
                    loss = criterion(outputs, targets.to(device))
                    tmp_test_loss.append(loss.detach())
                    _, predicted = outputs.max(1)
                    total += targets.size(0)
                    correct += predicted.eq(targets.to(device)).sum().detach() #.item() would transfer on CPU while .detach() works on GPU
            epochs_test_loss.append(torch.mean(torch.stack(tmp_test_loss)))
            accuracy = 100 * correct / total
            epochs_test_accuracy.append(accuracy)
            print('Accuracy of the model on the testing images: %f %%' % accuracy)
        
        correct = 0
        total = 0
        tmp_train_loss = []
        for (x, y) in trainloader:
            outputs = model(x.to(device))
            loss = criterion(outputs, y.to(device))
            tmp_train_loss.append(loss.detach())
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y.to(device)).sum().detach()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        epochs_train_loss.append(torch.mean(torch.stack(tmp_train_loss)))
        accuracy = 100 * correct / total
        epochs_train_accuracy.append(accuracy)
        print('Accuracy of the model on the training images: %f %%' % accuracy)

    return epochs_train_loss, epochs_test_loss, epochs_train_accuracy, epochs_test_accuracy

## Plot Losses and Accuracies

In [ ]:
def plot_history(train_loss = None, test_loss = None, train_accuracy = None, test_accuracy = None, validation = False):
    testing_string = ["Testing", "Validation"][validation]
    plt.figure(figsize=(14, 4))
    if train_loss and test_loss:
        plt.subplot(1, 2, 1)
        plt.title('Training Loss')
        plt.plot(train_loss)
        plt.xlabel('Epochs')
        plt.ylabel('(Cross Entropy) Loss')

        plt.subplot(1, 2, 2)
        plt.title(testing_string + ' Loss')
        plt.plot(epochs_test_loss)
        plt.xlabel('Epochs')
        plt.ylabel('(Cross Entropy) Loss')
    
    if train_accuracy and test_accuracy:
        plt.figure(figsize=(14, 4))
        plt.subplot(1, 2, 1)
        plt.title('Training Accuracy')
        plt.plot(epochs_train_accuracy)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')

        plt.subplot(1, 2, 2)
        plt.title(testing_string + ' Accuracy')
        plt.plot(epochs_test_accuracy)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')

## Train the Model (Train & Validation Sets)

In [ ]:
epochs_train_loss, epochs_validation_loss, epochs_train_accuracy, epochs_validation_accuracy = train(max_epochs, train_loader, validation_loader)

In [ ]:
plot_history(epochs_train_loss, epochs_validation_loss, epochs_train_accuracy, epochs_validation_accuracy)

## Train - Test Data Loader

In [ ]:
test_images, test_labels = load_dataset(root_dir, train=False)
print(f"Trainset shape: {len(full_train_set[0])}")
print(f"Test set shape: {len(test_images)}")

In [ ]:
full_train_set = GTSRBDataset(full_train_set[0], full_train_set[1], transform=train_transform)
full_train_loader = torch.utils.data.DataLoader(full_train_set, batch_size=100, shuffle=True, num_workers=50)

test_set = GTSRBDataset(test_images, test_labels, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=50)

## Train the Model (Train & Test Sets)

In [ ]:
epochs_train_loss, epochs_test_loss, epochs_train_accuracy, epochs_test_accuracy = train(num_epochs, train_loader, validation_loader)

In [ ]:
plot_history(epochs_train_loss, epochs_test_loss, epochs_train_accuracy, epochs_test_accuracy)